In [6]:
import random
import gym
import numpy as np
from collections import deque

In [7]:
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
from scores.score_logger import ScoreLogger
from sklearn.model_selection import train_test_split

In [8]:
#seu jogo de escolha
ENV_NAME = "CartPole-v1"
GAMMA = 0.95
LEARNING_RATE = 0.001
MEMORY_SIZE = 1000
BATCH_SIZE = 20
EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.05
EXPLORATION_DECAY = 0.96

In [44]:
##def cartpole
env = gym.make(ENV_NAME)
observation_space = env.observation_space.shape[0]
action_space = env.action_space.n

In [5]:
class DQNSolver:
    def __init__(self, observation_space, action_space):
        
        self.exploration_rate = EXPLORATION_MAX
        self.action_space = action_space
        self.memory = deque(maxlen = MEMORY_SIZE)        
        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_stage, done))
    
    def act(self, state):
        if np.random.rand()< exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])
    
    def exploration_rate(self):
        if len(self.memory)<BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                q_update =  (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose = 0)
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)

In [1]:
###########################################################################################

In [54]:
env = gym.make('MountainCarContinuous-v0') # try for different environements
env.action_space

Box(1,)

In [78]:
env.reset()

array([-0.03570481,  0.01183121,  0.0260376 ,  0.03959878])

In [1]:
# #https://gist.github.com/iambrian/2bcc8fc03eaecb2cbe53012d2f505465
# import gym
# env = gym.make('CartPole-v0')
# highscore = 0
# for i_episode in range(20): # run 20 episodes
#     observation = env.reset()
#     points = 0 # keep track of the reward each episode
#     while True: # run until episode is done
#         env.render()
#         action = 1 if observation[2] > 0 else 0 # if angle if positive, move right. if angle is negative, move left
#         observation, reward, done, info = env.step(action)
#         points += reward
#         if done:
#           if points > highscore: # record high score
#             highscore = points
#             break
            
# env.close()

In [103]:
import gym
env = gym.make('CartPole-v0')
for i_episode in range(20):
    observation = env.reset() # initial observation
    aux = 0
    for t in range(50):
        env.render()
        aux += 1
        print('{}.observation: {}'.format(aux, observation))
        action = env.action_space.sample()
        print('--ação:{}'.format(action))
        observation, reward, done, info = env.step(action)
        #ok onde está as condições de done?
        print('--observation:{}, reward:{}, done:{}, info:{}'.format(observation, reward, done, info))
        if done:
            print("\n")
            print("Episode finished after {} timesteps".format(t+1).upper())
            break
env.close()

In [102]:
env.step(action)

(array([ -9.13693864, -12.93749645,   4.91046668,  -2.18595879]),
 0.0,
 True,
 {})

In [1]:
#########################################################################################

In [3]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

In [4]:
env = gym.make('FrozenLake-v0')

In [42]:
action_space = env.action_space.n
state_space = env.observation_space.n

In [43]:
q_table = np.zeros((state_space, action_space))

In [72]:
num_episodes = 10000
max_steps = 150

learning_rate = 0.1
discount_rate = 0.95

exploration_rate = 0.98
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [75]:
reward_episodes_list = []

for episode in range(num_episodes):
    state = env.reset()
    #env.render()
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps):
        
        exploration_rate_threshold = random.uniform(0,1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()
        
        new_state, reward, done, info = env.step(action)
        
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) +\
        learning_rate*(reward + discount_rate*np.max(q_table[new_state, :]))
        
        state = new_state
        rewards_current_episode += reward
        
        if done == True:
            break
    
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate)* \
        np.exp(-exploration_decay_rate*episode)
    
    reward_episodes_list.append(rewards_current_episode)

reward_per_k = np.split(np.array(reward_episodes_list), num_episodes/1000)
count = 1000
print('--'*10)
for r in reward_per_k:
    print(count, ': ', str(sum(r/1000)))
    count += 1000
    
print('--'*10)
print(q_table)

--------------------
1000 :  0.04900000000000004
2000 :  0.19700000000000015
3000 :  0.3900000000000003
4000 :  0.5210000000000004
5000 :  0.5810000000000004
6000 :  0.5890000000000004
7000 :  0.6180000000000004
8000 :  0.6560000000000005
9000 :  0.5910000000000004
10000 :  0.5760000000000004
--------------------
[[0.18506404 0.15213222 0.13944287 0.15344793]
 [0.09012277 0.10476312 0.0880154  0.15664491]
 [0.16215522 0.09718711 0.09776034 0.10215043]
 [0.08761844 0.08485919 0.08073325 0.09676052]
 [0.21098478 0.14862339 0.08831195 0.12975429]
 [0.         0.         0.         0.        ]
 [0.06879404 0.05552519 0.15762491 0.05245866]
 [0.         0.         0.         0.        ]
 [0.11445532 0.21481454 0.20107123 0.25525668]
 [0.28174888 0.31323646 0.25846396 0.2022548 ]
 [0.37004345 0.22243588 0.14910161 0.1476606 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.32441187 0.30260033 0.43181019 0.30236237]
 [0.50391656 0.71419107 0.4